# Purple Cats

## Objective

This videogame aims to learn how a quantum circuit works and to do additions and subtractions from Draper's method, as we believe that with an application can help people learn about quantum computation.

## Context
In a laboratory dedicated to problems of quantum mechanics they have 5 cats as pets, some are green and others are purple. One of the research experiments were in several boxes called QFT, QFT$^{-1}$ and P with certain dephase, one day when they left the cats they forgot to keep the boxes in a place where the cats could not see them, consequently the cats got into the boxes and when the researchers arrived they realized that the cats were green. In this game we must consider the proper way to arrange the boxes in such a way that we find the purple cats in a specific porden since they only know the number of the cat by the collar they wear. 


![init_cats](videogame_Img/init_cat.png)


## Teorical Part

This porject works using the Draper idea of the adder, that using the Amplitude Amplification, The central idea behind amplitude amplification is to amplify the probability of a desired outcome occurring by performing a sequence of reflections. These reflections rotate the initial state closer towards a desired target state, often called a marked state, for this we use the Quantum Fourier Transform for identify the correct target state, with a specific order of a collection of $P(\lambda)$ gates


# Rules


At the time the cats were found, only two boxes QFT and QFT$^{-1}$ were not moved, the first one at the beginning and the second one at the end, as shown in the following figure.

![template.png](videogame_Img/template.png)


You have the following boxes in an unordered manner:

- 5 box with $P({\pi})$
- 4 box with $P(\frac{\pi}{2})$
- 3 box with $P(\frac{\pi}{4})$
- 2 box with $P(\frac{\pi}{8})$
- 1 box with $P(\frac{\pi}{16})$



Find the way to find the purple cats by following the structure , where the boxes must be arranged in a row order for the cats, it is also possible that not all boxes can be used.


You can select or not to put one or more boxes



## Hints

The researchers found the following 4 answers, with this information helping to find purple cats.



![ex_10000.png](videogame_Img/ex_10000.png)

![ex_01000.png](videogame_Img/ex_01000.png)

![ex_00100.png](videogame_Img/ex_00100.png)

![ex_00010.png](videogame_Img/ex_00010.png)

![ex_00001.png](videogame_Img/ex_00001.png)

### Code to choose the random value


Using the noise model methods of qiskit we can added some noise like i na real hardware and general a quantum circuit using Hadamards and Cnots gates to generate a random circuit. With the nosie model we cam  consider the noise in the gates X

In [8]:
import numpy as np

# Qiskit libraries 
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, execute, QuantumRegister, ClassicalRegister
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.circuit.library import QFT
#Qsharp libraries
from azure.quantum.qiskit import AzureQuantumProvider
import qsharp
import qsharp.experimental
qsharp.experimental.enable_noisy_simulation()
qsharp.experimental.get_noise_model

<function qsharp.experimental.get_noise_model()>

In [9]:
provider = AzureQuantumProvider(
  resource_id="/subscriptions/b1d7f7f8-743f-458e-b3a0-3e09734d716d/resourceGroups/aq-hackathons/providers/Microsoft.Quantum/Workspaces/aq-hackathon-01",
  location="East US"
)

In [29]:
# generate the circuti that do a QRNG
def random_number(): 

    qrng = qsharp.compile("""
    open Microsoft.Quantum.Arrays;
    open Microsoft.Quantum.Measurement;
        
    operation RandomNumber() : Result[] {
        use register = Qubit[5] {
            // Set qubits in superposition.
            for index in 0 .. 10  {
    
            ApplyToEachA(H, register);
            CNOT(register[0], register[1]);
            CNOT(register[0], register[2]);
            CNOT(register[0], register[3]);
            CNOT(register[0], register[4]);

            CNOT(register[1], register[2]);
            CNOT(register[1], register[3]);
            CNOT(register[1], register[4]);

            CNOT(register[2], register[3]);
            CNOT(register[3], register[4]);
            
            CNOT(register[3], register[4]);
            X(register[0]);
            }
            // Measure all qubits and return.
            return ForEach(MResetZ, register);
        }
    }
    """)
    s = ""
    for i in qrng.simulate():
        s = s+str(i)
    return s

### proof the random number

We can call the method random_number() and return a bitstring with the expect output, where 
- 0 is equal a green cat
- 1 is equal a purple cat

In [30]:
purple_cats_numer = random_number()
purple_cats_numer

'00100'

## Design the algorithm for the videogames

We following the idea of the Draper addition, is neccesary use the QFT and  the inverse QFT, and for generate the data we can use the gates P, this is

$$\begin{split}P(\lambda) =
    \begin{pmatrix}
        1 & 0 \\
        0 & e^{i\lambda}
    \end{pmatrix}\end{split}$$
    
    
Whit this in a certain order of  $\lambda$ is important consider the next expression 

$$e^{i 2 \pi / 2^{n}}$$


- if $n = 1$ we can obtain  $e^{i  \pi }$

- if $n = 2$ we can obtain  $e^{i  \frac{\pi} {2}}$

- if $n = 3$ we can obtain  $e^{i  \frac{\pi} {4}}$

- if $n = 4$ we can obtain  $e^{i  \frac{\pi} {8}}$

- if $n = 5$ we can obtain  $e^{i  \frac{\pi} {16}}$

and we only have 5 cats, so is the same values for the boxes

In [12]:
#convert  integer value to bin value for the quantum circuit
def add_value(qc,data_qubits,const): 
    #a = bin(const)[2:]
    #while len(a) < data_qubits:
    #    a = '0'+a
    a=const[::-1]
    list_a = [0]*data_qubits
    for i in range(data_qubits): 
        if a[i] =='1':
            k = 0
            for j in range(i,data_qubits):
                list_a[data_qubits-j-1] +=np.pi/float(2**(k)) 
                k+=1
            
    for i in range(data_qubits):
        if list_a[i] != 0:
            qc.p(list_a[i],i)
            k+=1


In [34]:
def qft(data_qubits: int, approximation_degree: int = 1):
    qr = QuantumRegister(data_qubits)
    qc = QuantumCircuit(qr)

    for target in reversed(range(data_qubits)):
        qc.h(target)
        for k, control in enumerate(reversed(range(target)), start=2):
            if k > (data_qubits - approximation_degree):
                continue
            qc.cp(2 * np.pi / (2 ** k), control, target)

    return qc

In [38]:
def find_purple_cats(pi,pi_2,pi_4,pi_8,pi_16,init_state=0):
    num_cats = 5
    quantum_cats = QuantumRegister(num_cats, "cats")
    purple_cats = ClassicalRegister(num_cats,"purple_cats")
    qc = QuantumCircuit(quantum_cats,purple_cats)

    len_pi = len(pi)
    len_pi_2 = len(pi_2)
    len_pi_4 = len(pi_4)
    len_pi_8 = len(pi_8)
    len_pi_16 = len(pi_16)

    qc.h(range(num_cats))
    
    if init_state != 0:
        add_value(qc,5,init_state)
    
    for i in range(len_pi-1,-1,-1):
        if pi[i]!= 0 :
            qc.p(pi[i]*np.pi,i)
            
    for i in range(len_pi_2-1,-1,-1):
        if pi_2[i]!= 0 :
            qc.p(pi_2[i]*np.pi/2,i)
            
    for i in range(len_pi_4-1,-1,-1):
        if pi_4[i]!= 0 :
            qc.p(pi_4[i]*np.pi/4,i)

            
    for i in range(len_pi_8-1,-1,-1):
        if pi_8[i]!= 0 :
            qc.p(pi_8[i]*np.pi/8,i)
            
            
    for i in range(len_pi_16-1,-1,-1):
        if pi_16[i]!= 0 :
            qc.p(pi_16[i]*np.pi/16,i)   
    
    qc = qc.compose(QFT(num_qubits=num_cats,inverse=True),quantum_cats)
    #qc.append(qft(5), quantum_cats)
    qc.measure(quantum_cats,purple_cats)
    return qc

# How to play 

## case 1:

if you want to modify the **first green cat** to a **purple cat** is equal to **|00000> to |00001>**, the configuration is posible with the following code:

```
pi[4] = 1
pi_2[3] = 1
pi_4[2] = 1
pi_8[1] = 1
pi_16[0] = 1

```

## case 2:

if you want to modify the **second green cat** to a **purple cat** is equal to **|00000> to |00010>**, the configuration is posible with the following code:

```
pi[3] = 1
pi_2[2] = 1
pi_4[1] = 1
pi_8[0] = 1

```


## case 3:

if you want to modify the **third green cat** to a **purple cat** is equal to **|00000> to |00100>**, the configuration is posible with the following code:

```
pi[2] = 1
pi_2[1] = 1
pi_4[0] = 1

```


## case 4:

if you want to modify the **fourth green cat** to a **purple cat** is equal to **|00000> to |01000>**, the configuration is posible with the following code:

```
pi[1] = 1
pi_2[0] = 1
```



## case 5:

if you want to modify the **fifth green cat** to a **purple cat** is equal to **|00000> to |10000>**, the configuration is posible with the following code:

```
pi[0] = 1
```

In [51]:
# init variables
pi = [0]*5
pi_2 = [0]*4
pi_4 = [0]*3
pi_8 = [0]*2
pi_16 = [0]


# select the values 



pi[2] = 1
pi_2[1] = 1
pi_4[0] = 1





qc = find_purple_cats(pi,pi_2,pi_4,pi_8,pi_16)
#qc.draw("mpl")

In [52]:
qc = find_purple_cats(pi,pi_2,pi_4,pi_8,pi_16)
counts = execute(qc,provider.get_backend('ionq.simulator')).result().get_counts()

sol=list(counts.keys())[0]

print("your answer is ",sol)
if purple_cats_numer == sol:
    print("You Got it")
else:
    print("Try again")

............your answer is  00100
You Got it


## using a ionq qpu

we compare the result with a real hardware

In [53]:
qc = find_purple_cats(pi,pi_2,pi_4,pi_8,pi_16)

target_basis = ['rx', 'ry', 'rz', 'h', 'cx']
decomposed = transpile(qc,
                       basis_gates=target_basis, 
                       optimization_level=0) 

qpu_backend = provider.get_backend("ionq.qpu")
qpu_job = qpu_backend.run(decomposed, shots=1024)
job_id = qpu_job.id()

# Get the job results (this method also waits for the Job to complete):
import operator 
result = qpu_job.result()
counts = {format(n, "03b"): 0 for n in range(8)}
counts.update(result.get_counts(qc))
sol= max(counts.items(), key=operator.itemgetter(1))[0]
print("your answer is ",sol)
if purple_cats_numer == sol:
    print("You Got it")
else:
    print("Try again")

..........................................................your answer is  00100
You Got it


## Another modality

There are two more considerations that if the purple cats are not the same as the ones we expect, then we must reconvert this, for this, we also have that the P-boxes can be positive or negative.

In [54]:
init_state = random_number()

print("Initial state for the cats are",init_state)
print("The objective value is ", purple_cats_numer)

Initial state for the cats are 10000
The objective value is  00100


consider the initial state  we need to find the combination for the purple cats output, sometimes is add or subs so one binary number

In [23]:
# init variables
pi = [0]*5
pi_2 = [0]*4
pi_4 = [0]*3
pi_8 = [0]*2
pi_16 = [0]


# select the values 


pi[3] = -1
pi_2[2] = -1
pi_4[1] = -1
pi_8[0] = -1





In [24]:
qc = find_purple_cats(pi,pi_2,pi_4,pi_8,pi_16,init_state)
counts = execute(qc,provider.get_backend('ionq.simulator')).result().get_counts()
sol=list(counts.keys())[0]

print("your answer is ",sol)
if purple_cats_numer == sol:
    print("You Got it")
else:
    print("Try again")

............your answer is  01110
Try again


# Authors
- Alberto Maldonado Romo
- Hanah Rahman
- Jesse Fonseca 
- Karime Mejia
